### Файл обработки оригильного датасета

In [ ]:
%%capture
!gdown 1AYZ3EM1toLX6_8Pq61i7rmEVz47l0tBn
!pip install wfdb
!unzip ptbxl

In [ ]:
import numpy as np
import pandas as pd
import wfdb
from tqdm import tqdm
import warnings
import ast
from collections import Counter
warnings.filterwarnings('ignore')

In [ ]:
meta_data = pd.read_csv('ptbxl/ptbxl_database.csv')
scp_codes = pd.read_csv('ptbxl/scp_statements.csv', index_col='Unnamed: 0')
scp_codes['label'] = list(scp_codes.index)

data = pd.DataFrame(columns=['filename_hr', 'ecg_id'])


data.filename_hr = meta_data.filename_hr
data.ecg_id = meta_data.ecg_id.apply(int)

In [ ]:
meta_data.scp_codes = meta_data.scp_codes.apply(lambda x: ast.literal_eval(x))

In [ ]:
scp_codes = scp_codes[scp_codes.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in scp_codes.index:
            tmp.append(scp_codes.loc[key].label)
    return list(set(tmp))

data['result_class'] = meta_data.scp_codes.apply(aggregate_diagnostic)

In [ ]:
classes = []
for i in data.result_class:
    classes += i

classes = dict(Counter(classes))

In [ ]:
ecg_ids = list(data.ecg_id)
for ecg in tqdm(ecg_ids):
    line = data[data.ecg_id==ecg].values.tolist()[0]
    diags = data[data.ecg_id==ecg].result_class.values[0]
    if len(diags) > 1:
        for i in range(1, len(diags)):
            new_value = line[:2]+[[diags[i]]]
            new_value[1] = line[1]+21837

            data.loc[data.ecg_id==ecg, 'result_class'] = [diags[0]]
            new_value = {k:v for k, v in zip(data.columns, new_value)}
            data = data.append(new_value, ignore_index=True)

100%|██████████| 21799/21799 [00:40<00:00, 543.59it/s]


In [ ]:
data = data[data.result_class.apply(lambda x: len(x) != 0)]
data.result_class = data.result_class.apply(lambda x: x[0] if type(x) == list else x)
data = data[data.result_class.isin(list(classes.values()))]

In [ ]:
label2idx = {k:list(classes.values()).index(k) for k in classes.values()}

In [ ]:
data.result_class = data.result_class.replace(label2idx)

In [ ]:
data.to_csv('ptblx_full_signals.csv', index=False)

In [ ]:
!mkdir ptbxl_full_signals
!mkdir ptbxl_full_signals/signals

In [ ]:
for i in tqdm(list(data.filename_hr)):
    hr = wfdb.rdsamp('ptbxl/'+i)[0].T
    np.save('ptbxl_full_signals/signals/'+i[-8:], hr)

100%|██████████| 27857/27857 [03:18<00:00, 140.38it/s]


In [ ]:
!zip -r ptbxl_full_signals.zip ptbxl_full_signals

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mv ptbxl_full_signals.zip /content/drive/MyDrive/cardioScreen/data/